# Stock Price Forecasting with Machine Learning

## 0. Preliminary workings

### 0.1 Uploading necessary packages

In [1]:
import os
import numpy as np
from scipy.stats import norm, t, uniform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from tqdm import tqdm
from datetime import datetime as dt

from linearmodels.panel import PanelOLS
import statsmodels.api as sm

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import HuberRegressor
from sklearn.cross_decomposition import PLSRegression
import time
import warnings
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import random
from scipy.optimize import minimize
from functools import partial
from numpy.linalg import svd
from sklearn.linear_model import ElasticNet
from group_lasso import GroupLasso
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

import random
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, BatchNormalization
from keras.regularizers import L1L2
from keras.optimizers import Adam
from dieboldmariano import dm_test

warnings.filterwarnings("ignore")
random.seed(123)

C:\Users\dguse\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dguse\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\dguse\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### 0.2.1 Defining few functions (may not be relevant anymore)

In [2]:
def fw1(x):
    # Find the maximum location of a vector
    maximum = np.max(x)
    p = np.where(x == maximum)[0]
    if len(p) > 1:
        p = p[0]
    return p

def pls(X, y, A):
    """
    Partial Least Squares (PLS) regression
    
    Parameters:
    X : array-like, shape (n_samples, n_features)
        Training data.
    y : array-like, shape (n_samples,)
        Target values.
    A : int
        Number of components.
        
    Returns:
    B : array, shape (n_features, A)
        Coefficients for each component.
    """
    # Initialize arrays to store intermediate results
    s = X.T.dot(y)
    R = np.zeros((X.shape[1], A))
    TT = np.zeros((X.shape[0], A))
    P = np.zeros((X.shape[1], A))
    U = np.zeros((X.shape[0], A))
    V = np.zeros((X.shape[1], A))
    B = np.zeros((X.shape[1], A))
    Q = np.zeros((1, A))

    # Iterate through each component
    for i in range(A):
        # Calculate loading vector for X and Y
        q = s.T.dot(s)
        r = s.dot(q)
        t = X.dot(r)
        t = t - np.mean(t)
        normt = np.sqrt(t.T.dot(t))
        t = t / normt
        r = r / normt
        p = X.T.dot(t)
        q = y.T.dot(t)
        u = y * q
        v = p

        # Calculate deflation
        if i > 0:
            v = v - V[:, :i+1].dot(V[:, :i+1].T.dot(p))
            u = u - TT[:, :i+1].dot(TT[:, :i+1].T.dot(u))
        v = v / np.sqrt(v.T.dot(v))
        s = s - v.dot(v.T.dot(s))

        # Store results for current component
        R[:, i] = r
        TT[:, i] = t
        P[:, i] = p
        U[:, i] = u
        V[:, i] = v
        Q[:, i] = q

    # Reconstruct the coefficient matrix B
    for i in range(A - 1):
        C = R[:, :i+1].dot(Q[:, :i+1].T)
        B[:, i+1] = C[:, 0]

    return B

def soft_threshodl(groups, nc, w, mu):
    """
    Soft thresholding operator
    
    Parameters:
    groups : int
        Not used in the function, placeholder for the MATLAB code.
    nc : int
        Not used in the function, placeholder for the MATLAB code.
    w : array-like
        Input array.
    mu : float
        Threshold parameter.
    
    Returns:
    val : array-like
        Soft thresholded array.
    """
    val = np.sign(w) * np.maximum(np.abs(w) - mu, 0)
    return val

def lossh(y, yhat, mu):
    """
    Loss function for proximalH
    
    Parameters:
    y : array-like
        True target values.
    yhat : array-like
        Predicted values.
    mu : float
        Threshold parameter.
    
    Returns:
    m : float
        Loss value.
    """
    r = np.abs(yhat - y)
    l = np.zeros(len(r))
    ind = (r > mu)
    l[ind] = 2 * mu * r[ind] - mu * mu
    ind = (r <= mu)
    l[ind] = r[ind] * r[ind]
    m = np.mean(l)
    return m

def f_gradh(w, X, y, mu):
    """
    Gradient of the loss function for proximalH
    
    Parameters:
    w : array-like
        Coefficients.
    X : array-like
        Training data.
    y : array-like
        Target values.
    mu : float
        Threshold parameter.
    
    Returns:
    grad : array-like
        Gradient.
    """
    r = np.dot(X, w) - y
    ind0 = np.where(np.abs(r) <= mu)[0]
    ind1 = np.where(r > mu)[0]
    indf1 = np.where(r < -mu)[0]
    grad = np.dot(X[ind0, :].T, np.dot(X[ind0, :], w) - y[ind0]) + mu * np.dot(X[ind1, :].T, np.ones(len(ind1))) - mu * np.dot(X[indf1, :].T, np.ones(len(indf1)))
    return grad

def proximalH(groups, nc, xtest, mtrain, ytest, w, X, y, mu, tol, L, l2, func):
    """
    Proximal operator using accelerated proximal gradient descent
    
    Parameters:
    groups : int
        Not used in the function, placeholder for the MATLAB code.
    nc : int
        Not used in the function, placeholder for the MATLAB code.
    xtest : array-like
        Test data.
    mtrain : float
        Mean of the training target values.
    ytest : array-like
        Test target values.
    w : array-like
        Initial guess of the coefficients.
    X : array-like
        Training data.
    y : array-like
        Target values.
    mu : float
        Threshold parameter.
    tol : float
        Tolerance parameter for convergence.
    L : float
        Lipschitz constant.
    l2 : float
        Regularization parameter.
    func : function
        Soft thresholding function.
    
    Returns:
    a : array-like
        Final coefficients after proximal gradient descent.
    """
    dim = X.shape[1]
    max_iter = 3000
    gamma = 1 / L
    l1 = l2
    v = w.copy()
    yhatbig1 = np.dot(xtest, w) + mtrain
    r20 = lossh(yhatbig1, ytest, mu)
    
    for t in range(max_iter):
        vold = v.copy()
        w_perv = w.copy()
        w = v - gamma * f_gradh(v, X, y, mu)
        mu1 = l1 * gamma
        w = func(groups, nc, w, mu1)
        v = w + t / (t + 3) * (w - w_perv)
        
        if np.sum((v - vold) ** 2) < (np.sum(vold ** 2) * tol) or np.sum(np.abs(v - vold)) == 0:
            break
    
    return v

def proximal(groups, nc, XX, XY, tol, L, l2, func):
    '''
    groups: Group membership of features
    nc: Number of classes
    XX: Matrix of features
    XY: Matrix of labels
    tol: Tolerance for convergence
    L: Lipschitz constant
    l2: L2 regularization parameter
    func: Proximal operator function
    '''
    dim = XX.shape[0]
    max_iter = 30000
    gamma = 1 / L
    l1 = l2
    w = np.zeros(dim)
    v = w.copy()

    for t in range(max_iter):
        vold = v.copy()
        w_prev = w.copy()
        w = v - gamma * f_grad(XX, XY, v)
        w = func(groups, nc, w, l1 * gamma)
        v = w + t / (t + 3) * (w - w_prev)
        if (np.sum(np.power(v - vold, 2)) < (np.sum(np.power(vold, 2)) * tol)) or (np.sum(np.abs(v - vold)) == 0):
            break

    return v

def f_grad(XX, XY, w):
    """
    Gradient of the objective function.

    Parameters:
    XX (array): Design matrix.
    XY (array): Target values.
    w (array): Coefficients.

    Returns:
    grad (array): Gradient.
    """
    grad = np.dot(XX, w) - XY
    return grad

def soft_threshodr(groups, nc, w, mu):
    """
    Soft thresholding function for ridge regularization.

    Parameters:
    groups (array): Groups.
    nc (int): Number of components.
    w (array): Coefficients.
    mu (float): Threshold parameter.

    Returns:
    val (array): Updated coefficients after soft thresholding.
    """
    val = w / (1 + mu)
    return val

import numpy as np

def cut_knots_degree2(x, n, th):
    '''
    x: Input matrix
    n: Degree of the polynomial
    th: Threshold values for knot points
    '''
    a, b = x.shape  # Get the dimensions of the input matrix
    resultfinal = np.zeros((a, b * (n + 1)))  # Initialize the final result matrix

    for i in range(b):
        xcut = x[:, i]  # Extract a column of the input matrix
        xcutnona = np.copy(xcut)  # Create a copy of the column
        xcutnona[np.isnan(xcutnona)] = 0  # Replace NaN values with 0
        index = ((1 - 1 * np.isnan(xcut)) == 1)  # Find non-NaN indices

        t = th[:, i]  # Get the threshold values for this column

        x1 = xcutnona
        resultfinal[:, (n + 1) * i - n] = x1 - np.mean(x1)  # Store the original values

        x1 = np.power(xcutnona - t[0], 2)
        resultfinal[:, (n + 1) * i - n + 1] = x1 - np.mean(x1)  # Store the squared differences with the first threshold

        for j in range(n - 1):
            x1 = np.power(xcutnona - t[j + 1], 2) * (xcutnona >= t[j + 1])  # Calculate squared differences with subsequent thresholds
            resultfinal[:, (n + 1) * i - n + 1 + j] = x1 - np.mean(x1)  # Store the result in the final matrix

    return resultfinal

def soft_threshode(groups, nc, w, mu):
    '''
    groups: Group membership of features
    nc: Number of classes
    w: Input vector
    mu: Threshold parameter
    '''
    # Apply soft thresholding operation element-wise
    val = np.sign(w) * np.maximum(np.abs(w) - 0.5 * mu, 0) / (1 + 0.5 * mu)
    
    return val

def soft_threshodg(groups, nc, w, mu):
    '''
    groups: Group membership of features
    nc: Number of classes
    w: Input vector
    mu: Threshold parameter
    '''
    w1 = np.copy(w)  # Create a copy of the input vector
    for i in range(1, nc + 1):  # Iterate over the number of classes
        ind = (groups == i)  # Identify indices corresponding to the current class
        wg = w1[ind]  # Extract the values of w corresponding to the current class
        nn = len(wg)  # Get the number of elements in the current class
        n2 = np.sqrt(np.sum(wg ** 2))  # Calculate the L2 norm of the current class
        if n2 <= mu:  # Check if the L2 norm is less than or equal to the threshold parameter mu
            w1[ind] = np.zeros(nn)  # Set the values of w corresponding to the current class to zero
        else:
            w1[ind] = wg - mu * wg / n2  # Apply the soft thresholding operation to the values of w corresponding to the current class
    return w1  # Return the updated vector w

### 0.2.2. New set of functions

In [3]:
# Loss Function
# Huber objective function
def huber(actual, predicted, xi):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    resid = actual - predicted
    huber_loss = np.where(np.abs(resid)<=xi, diff**2, 2*xi*np.abs(resid)-xi**2)
    return np.mean(huber_loss)

# Scoring Function
# out-of-sample R squared
def R_oos(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted).flatten()
    predicted = np.where(predicted<0,0,predicted)
    return 1 - (np.dot((actual-predicted),(actual-predicted)))/(np.dot(actual,actual))

# Validation Function
def val_fun(model, params: dict, X_trn, y_trn, X_vld, y_vld, illustration=True, sleep=0, is_NN=False):
    best_ros = None
    lst_params = list(ParameterGrid(params))
    for param in lst_params:
        if best_ros == None:
            if is_NN:
                mod = model().set_params(**param).fit(X_trn, y_trn, X_vld, y_vld)
            else:
                mod = model().set_params(**param).fit(X_trn, y_trn)
            best_mod = mod
            y_pred = mod.predict(X_vld)
            best_ros = R_oos(y_vld, y_pred)
            best_param = param
            if illustration:
                print(f'Model with params: {param} finished.')
                print(f'with out-of-sample R squared on validation set: {best_ros*100:.5f}%')
                print('*'*60)
        else:
            #time.sleep(sleep)
            if is_NN:
                mod = model().set_params(**param).fit(X_trn, y_trn, X_vld, y_vld)
            else:
                mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            ros = R_oos(y_vld, y_pred)
            if illustration:
                print(f'Model with params: {param} finished.')
                print(f'with out-of-sample R squared on validation set: {ros*100:.5f}%')
                print('*'*60)
            if ros > best_ros:
                best_ros = ros
                best_mod = mod
                best_param = param
    if illustration:
        print('\n'+'#'*60)
        print('Tuning process finished!!!')
        print(f'The best setting is: {best_param}')
        print(f'with R2oos {best_ros*100:.2f}% on validation set.')
        print('#'*60)
    return best_mod
    

# Pairwise Comparison
# Diebold-Mariano test statistics

# Evaluation Output
def evaluate(actual, predicted, insample=False):
    if insample:
        print('*'*15+'In-Sample Metrics'+'*'*15)
        print(f'The in-sample R2 is {r2_score(actual,predicted)*100:.2f}%')
        print(f'The in-sample MSE is {mean_squared_error(actual,predicted):.3f}')
    else:
        print('*'*15+'Out-of-Sample Metrics'+'*'*15)
        print(f'The out-of-sample R2 is {R_oos(actual,predicted)*100:.2f}%')
        print(f'The out-of-sample MSE is {mean_squared_error(actual,predicted):.3f}')

In [4]:
class PCRegressor:
    def __init__(self,n_PCs=1,loss='mse'):
        self.n_PCs = n_PCs
        if loss not in ['huber','mse']:
            raise AttributeError(
            f"The loss should be either 'huber' or 'mse', but {loss} is given"
            )
        else:
            self.loss = loss
        
    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
        
    def fit(self,X,y):
        X = np.array(X)
        N,K = X.shape
        y = np.array(y_trn).reshape((N,1))
        self.mu = np.mean(X,axis=0).reshape((1,K))
        self.sigma = np.std(X,axis=0).reshape((1,K))
        self.sigma = np.where(self.sigma==0,1,self.sigma)
        X = (X-self.mu)/self.sigma
        pca = PCA()
        X = pca.fit_transform(X)[:,:self.n_PCs]
        self.pc_coef = pca.components_.T[:,:self.n_PCs]
        if self.loss == 'mse':
            self.model = LinearRegression().fit(X,y)
        else:
            self.model = HuberRegressor().fit(X,y)
        return self
    
    def predict(self,X):
        X = np.array(X)
        X = (X-self.mu)/self.sigma
        X = X @ self.pc_coef
        return self.model.predict(X)
    
    def get_params(self, X,y):
        X = np.array(X)
        N,K = X.shape
        y = np.array(y_trn).reshape((N,1))
        self.mu = np.mean(X,axis=0).reshape((1,K))
        self.sigma = np.std(X,axis=0).reshape((1,K))
        self.sigma = np.where(self.sigma==0,1,self.sigma)
        X = (X-self.mu)/self.sigma
        pca = PCA()
        X = pca.fit_transform(X)[:,:self.n_PCs]
        self.pc_coef = pca.components_.T[:,:self.n_PCs]
        return self.pc_coef

In [5]:
# mse
def mse(actual, predicted):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    resid = actual - predicted
    return np.mean(resid**2)

# huber objective function
def huber(actual, predicted, xi):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    resid = actual - predicted
    huber_loss = np.where(np.abs(resid)<=xi, diff**2, 2*xi*np.abs(resid)-xi**2)
    return np.mean(huber_loss)

# proximal operator
def prox(theta,lmd,rho,gamma):
    return (1/(1+lmd*gamma*rho))*softhred(theta,(1-rho)*gamma*lmd)

# soft-thresholding operator
def softhred(x,mu):
    x = np.where(np.abs(x)<=mu, 0, x)
    x = np.where((np.abs(x)>mu) & (x>0), x-mu, x)
    x = np.where((np.abs(x)>mu) & (x<0), x+mu, x)
    return x

# penalized mse
def mse_pnl(theta, X, y, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    resid = y - X@theta.reshape((K,1))
    return np.mean(resid**2) + lmd*(1-rho)*np.sum(np.abs(theta)) + 0.5*lmd*rho*np.sum(theta**2)

# penalized huber objective function
def huber_pnl(theta, X, y, xi, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    resid = y - X@theta.reshape((K,1))
    huber_loss = np.where(np.abs(resid)<=xi, resid**2, 2*xi*np.abs(resid)-xi**2)
    return np.mean(huber_loss) + lmd*(1-rho)*np.sum(np.abs(theta)) + 0.5*lmd*rho*np.sum(theta**2)

# gradient of mse
def grad_mse(theta, X, y, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    theta = np.array(theta).reshape((K,1))
    grad = (X.T @ (y - X@theta))/N + lmd*(1-rho)*np.where(theta>0,1,-1) + lmd*rho*theta
    return grad.flatten()

# gradient of huber loss
def grad_huber(theta, X, y, xi, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    theta = np.array(theta).reshape((K,1))
    resid = y - X@theta
    ind_m = np.where(np.abs(resid)<=xi)
    ind_u = np.where(resid>xi)
    ind_l = np.where(resid< -xi)
    try:
        grad_m = X[ind_m].T @ (y[ind_m] - X[ind_m]@theta)
    except:
        grad_m = np.zeros((K,1))
    try:
        grad_u = 2*xi* X[ind_u].T@np.ones((len(ind_u[0]),1))
    except:
        grad_u = np.zeros((K,1))
    try:
        grad_l = -2*xi* X[ind_l].T@np.ones((len(ind_l[0]),1))
    except:
        grad_l = np.zeros((K,1))
    grad = (grad_m+grad_u+grad_l)/N + lmd*(1-rho)*np.where(theta>0,1,-1) + lmd*rho*theta
    return grad.flatten()

# Elastic Net
class ENet:
    
    def __init__(
        self, lmd, rho, xi=1.35, loss='huber', random_state=None, fit_intercept=True
    ):
        self.lmd = lmd
        self.rho = rho
        self.xi = xi
        self.random_state = random_state
        self.fit_intercept = fit_intercept
        if loss not in ['huber','mse']:
            raise AttributeError(
            f"The loss should be either 'huber' or 'mse', but {loss} is given"
            )
        else:
            self.loss = loss
            
    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
            
    def fit(self, X, y):
        K = X.shape[1]
        X = np.array(X)
        N = len(y)
        y = np.array(y).reshape((N,1))
        if self.fit_intercept:
            K += 1
            X = np.concatenate((np.ones((N,1)),X),axis=1)
        # initialize theta
        if self.random_state != None:
            np.random.seed(self.random_state)
            theta = np.random.uniform(K)
        else:
            theta = np.zeros(K)
        
        if self.loss == 'huber':
            res = minimize(
                partial(huber_pnl, X=X, y=y, xi=self.xi, lmd=self.lmd, rho=self.rho), theta,
                method='nelder-mead',
                #method='BFGS',
                #jac = partial(grad_huber, X=X, y=y, xi=self.xi, lmd=self.lmd, rho=self.rho),
                options = {'disp': True}
            )
        else:
            res = minimize(
                partial(mse_pnl, X=X, y=y, lmd=self.lmd, rho=self.rho), theta, 
                method='nelder-mead',
                #method='BFGS',
                #jac = partial(grad_mse, X=X, y=y, lmd=self.lmd, rho=self.rho),
                options = {'disp': True}
            )
        
        self.theta = res.x.reshape((K,1))
        return self
    
    def predict(self, X):
        X = np.array(X)
        N = X.shape[0]
        if self.fit_intercept:
            X = np.concatenate((np.ones((N,1)),X),axis=1)
        return X@self.theta
    
    def get_params():
        return self

In [6]:
def flatten(l):
    return [item for sublist in l for item in sublist]

def SplineTransform(data,knots=3):
    spline_data = pd.DataFrame(np.ones((data.shape[0],1)),index=data.index,columns=['const'])
    for i in data.columns:
        i_dat = data.loc[:,i]
        i_sqr = i_dat**2
        i_cut, bins = pd.cut(i_dat, 3, right=True, ordered=True, retbins=True)
        i_dum = pd.get_dummies(i_cut)
        for j in np.arange(knots):
            i_dum.iloc[:,j] = i_dum.iloc[:,j]*((i_dat-bins[j])**2)
        i_dum.columns = [f"{i}_{k}" for k in np.arange(1,knots+1)]
        spline_data = pd.concat((spline_data,i_dat,i_dum),axis=1)
    return spline_data

class GLMRegression:
    
    def __init__(self,knots=3,lmd=1e-4,l1_reg=1e-4,random_state=12308):
        self.knots = knots
        self.lmd = lmd
        self.random_state = random_state
        self.l1_reg = l1_reg
        
    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
    
    def fit(self,X,y):
        groups = [0]+flatten([list(np.repeat(i,self.knots+1))[:] for i in np.arange(1,X.shape[1]+1)])
        X = SplineTransform(X)
        self.mod = GroupLasso(
            groups=groups,group_reg=self.lmd,l1_reg=self.l1_reg,
            fit_intercept=False,random_state=self.random_state
        )
        self.mod = self.mod.fit(X,y)
        return self
    
    def predict(self,X):
        X = SplineTransform(X)
        return self.mod.predict(X)
    
    def get_params(self,X,y):
        groups = [0]+flatten([list(np.repeat(i,self.knots+1))[:] for i in np.arange(1,X.shape[1]+1)])
        X_trans = SplineTransform(X)
        indexes = [X_trans.columns.get_loc(col) for col in X.columns]
        self.mod = GroupLasso(
            groups=groups,group_reg=self.lmd,l1_reg=self.l1_reg,
            fit_intercept=False,random_state=self.random_state
        )
        coeffis = self.mod.fit(X_trans,y).coef_
        self.good = [coeffis[i] for i in indexes]
        return self.good

### 0.3 Merging datasets

In [ ]:
c1 = pd.read_csv('datashare.csv')
r1 = pd.read_csv('funny_data.csv')
some = pd.read_csv('small.csv')

In [ ]:
c1['DATE'] = pd.to_datetime(c1['DATE'],format='%Y%m%d')+pd.offsets.MonthEnd(0)
r1['DATE'] = r1['date']
r1['DATE'] = pd.to_datetime(r1['DATE'])+pd.offsets.MonthEnd(0)

In [ ]:
c1.to_csv('./Simu/Simu_p50/c1_1.csv', index=False)
r1.to_csv('./Simu/Simu_p50/r1_1.csv', index=False)

In [7]:
c1 = pd.read_csv("./Simu/Simu_p50/c1_1.csv")
r1 = pd.read_csv("./Simu/Simu_p50/r1_1.csv")

In [8]:
stock = pd.Categorical(c1['permno'])
c1 = c1.set_index(["permno", "DATE"])
c1['stock'] = stock

In [9]:
stock = pd.Categorical(r1["PERMNO"])
r1 = r1.set_index(["PERMNO", "date"])
r1['stock'] = stock

In [10]:
c1.index.names = ['permno', 'date']
r1.index.names = ['permno', 'date']

In [11]:
c1 = c1.sort_index(level=['permno', 'date'])
r1 = r1.sort_index(level=['permno', 'date'])

In [12]:
new_date_index = pd.to_datetime(r1.index.get_level_values(level = "date"))
new_index = pd.MultiIndex.from_tuples(zip(r1['stock'], new_date_index), names=['permno', 'date'])
r1.index = new_index

new_date_index = pd.to_datetime(c1.index.get_level_values(level = "date"))
new_index = pd.MultiIndex.from_tuples(zip(c1['stock'], new_date_index), names=['permno', 'date'])
c1.index = new_index

---------------------

#### Set the prediod of prediction

In [13]:
starting = pd.to_datetime('2018-01-01')
ending = pd.to_datetime('2022-01-01')

In [14]:
c1_small = c1[(c1.index.get_level_values(level = "date") >= starting) & (c1.index.get_level_values(level = "date") <= ending)]
r1_small = r1[(r1.index.get_level_values(level = "date") >= starting) & (r1.index.get_level_values(level = "date") <= ending)]

---------------

In [15]:
b1 = c1_small.merge(r1_small, left_index=True, right_on=['permno', 'date'])

### 0.3.1 Cleaning the data from bad tickers

In [16]:
b1['TICKER'] = pd.Categorical(b1['TICKER'])
additional = pd.DataFrame(b1['TICKER'].value_counts())
bad_spisok = additional[additional['count'] <= 5].index

b1 = b1[~b1['TICKER'].isin(bad_spisok)]

In [17]:
bad_spisok

CategoricalIndex(['CHAA', 'ACQR', 'BRSP', 'AAQC', 'ATVC', 'ACND', 'ACBA',
                  'AADR', 'CIU', 'ZIP',
                  ...
                  'CCRD', 'SPCX', 'PEAR', 'WULF', 'GUG', 'LCW', 'SARK', 'CDRE',
                  'WYIG', 'MYNA'],
                 categories=['A', 'AA', 'AAAP', 'AABA', ..., 'ZY', 'ZYME', 'ZYNE', 'ZYXI'], ordered=False, dtype='category', name='TICKER', length=1362)

### 0.4 Filling missing values

In [18]:
b1 = b1.drop(columns = ['stock_x', 'stock_y', 'DATE', 'TICKER', 'RETX'])
b1['RET'] = pd.to_numeric(b1['RET'], errors = "coerce")

characteristics = list(set(b1.columns).difference({'SHROUT','mve0','sic2','prc', "SICCD"}))
b1.isnull().sum()

mvel1         44
beta       14573
betasq     14573
chmom      12642
dolvol      1356
           ...  
ASKHI          0
PRC          689
VOL            0
RET          690
SPREAD    189624
Length: 104, dtype: int64

In [19]:
for index, value in enumerate(b1['RET']):
    if isinstance(value, str):
        b1.at[index, 'RET'] = np.nan

In [20]:
%%time
# fill na with cross-sectional median
for ch in characteristics:
     b1[ch] = b1.groupby('date')[ch].transform(lambda x: x.fillna(x.median()))

CPU times: total: 3.66 s
Wall time: 3.65 s


In [21]:
for ch in characteristics:
     b1[ch] = b1[ch].fillna(0)
    
b1.columns[b1.isnull().sum()!=0]

Index(['sic2'], dtype='object')

### 0.5 Adjusting Industrical Classification variable 

In [22]:
def get_sic_dummies(data_ch):
    sic_dummies = pd.get_dummies(b1['sic2'].fillna(999).astype(int),prefix='sic').drop('sic_999',axis=1)
    b1_d = pd.concat([b1,sic_dummies],axis=1)
    b1_d.drop(['sic2'],inplace=True,axis=1)
    b1_d.drop(['SICCD'],inplace=True,axis=1)
    return b1_d

In [23]:
b1_d = get_sic_dummies(b1)
b1_d.dtypes.value_counts()

float64    101
bool        68
int64        1
Name: count, dtype: int64

### 0.6 Train-Test Split

In [24]:
from sklearn.preprocessing import MinMaxScaler

index_co = b1_d.index
features = list(set(b1_d.columns).difference({'permno','DATE','RET'}))
X = MinMaxScaler((-1,1)).fit_transform(b1_d[features])
X = pd.DataFrame(X, columns=features, index=index_co)
y = b1_d['RET']

In [25]:

stdt_vld = starting + (ending - starting)/3
stdt_tst = starting + 2*(ending - starting)/3

X_trn = X[X.index.get_level_values(level = "date") <= stdt_vld]
y_trn = y[y.index.get_level_values(level = "date") <= stdt_vld]

X_vld = X[(X.index.get_level_values(level = "date") >= stdt_vld) & (X.index.get_level_values(level = "date") <= stdt_tst)]
y_vld = y[(y.index.get_level_values(level = "date") >= stdt_vld) & (y.index.get_level_values(level = "date") <= stdt_tst)]

X_tst = X[X.index.get_level_values(level = "date") >= stdt_tst]
y_tst = y[y.index.get_level_values(level = "date") >= stdt_tst]

-----------------------

## 1.0 Linear Model (Old)

In [26]:
# PooledOLS - this is what I need to do. Plus add the Huber Func
exog_vars = list(set(b1.columns).difference({'sic2','RET', "SICCD"}))
exog = sm.add_constant(b1[exog_vars])
mod = PanelOLS(b1.RET, exog, check_rank=False)
pooled_res = mod.fit()
print(pooled_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                    RET   R-squared:                        0.0308
Estimator:                   PanelOLS   R-squared (Between):             -0.3514
No. Observations:              192242   R-squared (Within):               0.0434
Date:                Wed, Apr 24 2024   R-squared (Overall):              0.0308
Time:                        08:02:17   Log-likelihood                 3.695e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      60.555
Entities:                        7177   P-value                           0.0000
Avg Obs:                       26.786   Distribution:              F(101,192140)
Min Obs:                       1.0000                                           
Max Obs:                       33.000   F-statistic (robust):             60.555
                            

In [27]:
# Plot some stocks and check how they behave. Check the data consostency and clear bad stocks (write them down in additional list.
# Create a table with each bad ticker and associated problem (and check the paper for the advice)

## 1.1 Linear Models (New)

### 1.1.1 OLS

In [28]:
%%time
### Simple OLS with MSE as a loss function

# OLS with all features
OLS = LinearRegression().fit(X_trn,y_trn)
evaluate(y_trn, OLS.predict(X_trn), insample=True)
evaluate(y_tst, OLS.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 2.77%
The in-sample MSE is 0.029
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 1.28%
The out-of-sample MSE is 0.050
CPU times: total: 6.06 s
Wall time: 1.13 s


In [29]:
%%time
### OLS with a Huber loss function

# OLS by Huber robust objective function with all features
epsilon = np.max(((y_trn-OLS.predict(X_trn)).quantile(.999),1))
OLS_H = HuberRegressor(epsilon=epsilon).fit(X_trn,y_trn)
evaluate(y_trn, OLS_H.predict(X_trn), insample=True)
evaluate(y_tst, OLS_H.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 1.65%
The in-sample MSE is 0.029
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.83%
The out-of-sample MSE is 0.050
CPU times: total: 19.7 s
Wall time: 9.61 s


In [30]:
# OLS with preselected size, bm, and momentum covariates
features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']
OLS_3 = LinearRegression().fit(X_trn[features_3],y_trn)

### 1.1.2 PLS

In [31]:
%%time

params = {'n_components': [1, 5, 10, 50]}
PLS = val_fun(PLSRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'n_components': 1} finished.
with out-of-sample R squared on validation set: 1.21289%
************************************************************
Model with params: {'n_components': 5} finished.
with out-of-sample R squared on validation set: 2.55827%
************************************************************
Model with params: {'n_components': 10} finished.
with out-of-sample R squared on validation set: 2.25932%
************************************************************
Model with params: {'n_components': 50} finished.
with out-of-sample R squared on validation set: 2.43065%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'n_components': 5}
with R2oos 2.56% on validation set.
############################################################
CPU times: total: 13.2 s
Wall time: 6.53 s


In [32]:
%%time
pls_pred_is = PLS.predict(X_trn)
pls_pred_os = PLS.predict(X_tst)
evaluate(y_trn, pls_pred_is, insample=True) 
evaluate(y_tst, pls_pred_os)

***************In-Sample Metrics***************
The in-sample R2 is 2.48%
The in-sample MSE is 0.029
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 1.32%
The out-of-sample MSE is 0.049
CPU times: total: 219 ms
Wall time: 138 ms


### 1.1.3 PCR

In [33]:
%%time

params = {'n_PCs':[1,3,5,7,10,50],'loss':['mse','huber']}
PCR = val_fun(PCRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,sleep=3)

Model with params: {'loss': 'mse', 'n_PCs': 1} finished.
with out-of-sample R squared on validation set: -0.08341%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 3} finished.
with out-of-sample R squared on validation set: -0.08293%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 5} finished.
with out-of-sample R squared on validation set: -0.03206%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 7} finished.
with out-of-sample R squared on validation set: -0.00721%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 10} finished.
with out-of-sample R squared on validation set: 0.28307%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 50} finished.
with out-of-sample R squared on validation set: 1.23369%
*****

In [34]:
%%time
evaluate(y_trn, PCR.predict(X_trn), insample=True) 
evaluate(y_tst, PCR.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 0.86%
The in-sample MSE is 0.030
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -0.40%
The out-of-sample MSE is 0.042
CPU times: total: 1.06 s
Wall time: 316 ms


### 1.1.4 Elastic Net

In [35]:
%%time
random.seed(12308)
EN_my_mse_rdm = ENet(lmd=.01,rho=.5,loss='huber').fit(X_trn,y_trn)

Optimization terminated successfully.
         Current function value: 0.023946
         Iterations: 6547
         Function evaluations: 8658
CPU times: total: 9min 3s
Wall time: 4min 44s


In [36]:
%%time
evaluate(y_trn, EN_my_mse_rdm.predict(X_trn), insample=True) 
evaluate(y_tst, EN_my_mse_rdm.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is -0.07%
The in-sample MSE is 0.030
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.03%
The out-of-sample MSE is 0.041
CPU times: total: 141 ms
Wall time: 123 ms


### 1.1.5 GLM

In [37]:
%%time

params = {
    'knots':[3],
    'lmd':[1e-4,1e-1],#list(np.linspace(1e-4,1e-1,10)),
    'l1_reg':[1e-4,0]
}
GLM = val_fun(GLMRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'knots': 3, 'l1_reg': 0.0001, 'lmd': 0.0001} finished.
with out-of-sample R squared on validation set: 0.15672%
************************************************************
Model with params: {'knots': 3, 'l1_reg': 0.0001, 'lmd': 0.1} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'knots': 3, 'l1_reg': 0, 'lmd': 0.0001} finished.
with out-of-sample R squared on validation set: 0.21839%
************************************************************
Model with params: {'knots': 3, 'l1_reg': 0, 'lmd': 0.1} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'knots': 3, 'l1_reg': 0, 'lmd': 0.0001}
with R2oos 0.22% on validation set.
#########################################################

In [38]:
%%time
evaluate(y_trn, GLM.predict(X_trn), insample=True) 
evaluate(y_tst, GLM.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 0.24%
The in-sample MSE is 0.030
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.46%
The out-of-sample MSE is 0.040
CPU times: total: 54.8 s
Wall time: 54.6 s


In [39]:
output = pd.DataFrame(columns = ["OLS + H", "OLS-3 + H", "PCR", "PLS", "Enet + H", "GLM + H", "RF", "GBRT + H"], data=np.zeros((3,8)))
output.index = ["All", "Top 1000", "Bottom 1000"]
output.iloc[0,0] =r2_score(y_tst, OLS_H.predict(X_tst))
output.iloc[0,2] = r2_score(y_tst, PCR.predict(X_tst))
output.iloc[0,3] = r2_score(y_tst, PLS.predict(X_tst))
output.iloc[0,4] = r2_score(y_tst, EN_my_mse_rdm.predict(X_tst))
output.iloc[0,5] = r2_score(y_tst, GLM.predict(X_tst))

output.iloc[0,1] = r2_score(y_tst, OLS_3.predict(X_tst[features_3]))

### Diebold-Mariano tests

In [40]:
moriano = pd.DataFrame(columns = ["OLS-3 + H", "PLS", "PCR", "Enet + H", "GLM + H", "RF", "GBRT + H", "NN1", "NN2", "NN3", "NN4", "NN5"], index = ["OLS + H", "OLS-3 + H", "PLS", "PCR", "Enet + H", "GLM + H", "RF", "GBRT + H", "NN1", "NN2", "NN3", "NN4"], data=np.zeros((12,12)))

In [41]:
moriano.iloc[0,0] = dm_test(y_tst.values, OLS_H.predict(X_tst), OLS_3.predict(X_tst[features_3]), one_sided = False)[0]
moriano.iloc[0,1] = dm_test(y_tst.values, OLS_H.predict(X_tst), PLS.predict(X_tst), one_sided = False)[0]
moriano.iloc[0,2] = dm_test(y_tst.values, OLS_H.predict(X_tst), PCR.predict(X_tst), one_sided = False)[0]
moriano.iloc[0,3] = dm_test(y_tst.values, OLS_H.predict(X_tst), EN_my_mse_rdm.predict(X_tst), one_sided = False)[0]
moriano.iloc[0,4] = dm_test(y_tst.values,OLS_H.predict(X_tst), GLM.predict(X_tst), one_sided = False)[0]

moriano.iloc[1,1] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), PLS.predict(X_tst), one_sided = False)[0]
moriano.iloc[1,2] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), PCR.predict(X_tst), one_sided = False)[0]
moriano.iloc[1,3] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), EN_my_mse_rdm.predict(X_tst), one_sided = False)[0]
moriano.iloc[1,4] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), GLM.predict(X_tst), one_sided = False)[0]

moriano.iloc[2,2] = dm_test(y_tst.values, PLS.predict(X_tst), PCR.predict(X_tst), one_sided = False)[0]
moriano.iloc[2,3] = dm_test(y_tst.values, PLS.predict(X_tst), EN_my_mse_rdm.predict(X_tst), one_sided = False)[0]
moriano.iloc[2,4] = dm_test(y_tst.values, PLS.predict(X_tst), GLM.predict(X_tst), one_sided = False)[0]

moriano.iloc[3,3] = dm_test(y_tst.values, PCR.predict(X_tst), EN_my_mse_rdm.predict(X_tst), one_sided = False)[0]
moriano.iloc[3,4] = dm_test(y_tst.values, PCR.predict(X_tst), GLM.predict(X_tst), one_sided = False)[0]

moriano.iloc[4,4] = dm_test(y_tst.values, EN_my_mse_rdm.predict(X_tst), GLM.predict(X_tst), one_sided = False)[0]

In [42]:
moriano

,OLS-3 + H,PLS,PCR,Enet + H,GLM + H,RF,GBRT + H,NN1,NN2,NN3,NN4,NN5
OLS + H,44.667607,13.617104,50.423308,47.538610,47.408230,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OLS-3 + H,0.000000,-37.000430,-6.857542,21.822947,24.353913,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PLS,0.000000,0.000000,42.498729,41.597149,41.618567,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PCR,0.000000,0.000000,0.000000,20.385630,21.651692,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Enet + H,0.000000,0.000000,0.000000,0.000000,17.784107,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GLM + H,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RF,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GBRT + H,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NN1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NN2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Coefficient Significance

In [43]:
significo = pd.DataFrame(columns = ["PLS", "PCR", "Enet + H", "GLM + H", "RF", "GBRT + H", "NN1", "NN2", "NN3", "NN4", "NN5"], index = X_tst.columns, data=np.zeros((169,11)))

In [44]:
significo["PLS"] = PLS.coef_

joe = PCRegressor()
significo["PCR"] = joe.get_params(X_trn, y_trn).flatten()

#significo["Enet + H"] = EN_my_mse_rdm.coef_

joe = GLMRegression()
flattened_list = []
significo["GLM + H"] = joe.get_params(X_trn,y_trn)

In [45]:
significo

,PLS,PCR,Enet + H,GLM + H,RF,GBRT + H,NN1,NN2,NN3,NN4,NN5
pchgm_pchsale,-0.002017,-0.058591,0.0,[0.0],0.0,0.0,0.0,0.0,0.0,0.0,0.0
sic_80,0.000681,-0.000182,0.0,[0.0],0.0,0.0,0.0,0.0,0.0,0.0,0.0
sic_55,0.000914,-0.010684,0.0,[0.0],0.0,0.0,0.0,0.0,0.0,0.0,0.0
currat,0.000368,0.036253,0.0,[0.0],0.0,0.0,0.0,0.0,0.0,0.0,0.0
sgr,-0.000459,0.052028,0.0,[-0.0],0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
sic_81,0.000663,-0.002312,0.0,[-0.0],0.0,0.0,0.0,0.0,0.0,0.0,0.0
ill,0.004975,0.014122,0.0,[0.0],0.0,0.0,0.0,0.0,0.0,0.0,0.0
sic_49,0.000419,-0.026146,0.0,[0.0],0.0,0.0,0.0,0.0,0.0,0.0,0.0
age,-0.001054,-0.116730,0.0,[0.0],0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1.2 Ensemble Models

### 1.2.0 Additional Functions

In [46]:
# huber loss function for customized objective function

# gradient of huber loss with respect to y_pred
def grad_huber_obj(y_true, y_pred):
    xi = 1.35 
    # Though I do not want to make it hard-coded, lightgbm, behind the scene, evaluates the # of parameters
    # of the objective function first, then pass according # of parameters. I tried to use partial to set 
    # the value of xi. It did not work.
    # I refer the readers to the source code to have a better understanding of the issue:
    # (https://github.com/microsoft/LightGBM/blob/master/python-package/lightgbm/sklearn.py)
    y_true, y_pred = np.array(y_true).flatten(), np.array(y_pred).flatten()
    N = len(y_true)
    resid = y_true - y_pred
    ind_m = np.where(np.abs(resid)<=xi)
    ind_u = np.where(resid>xi)
    ind_l = np.where(resid< -xi)
    grad = np.zeros(N)
    try:
        grad[ind_m] = (-2*(y_true-y_pred))[ind_m]
    except:
        pass
    try:
        grad[ind_u] = np.repeat(2*xi,N)[ind_u]
    except:
        pass
    try:
        grad[ind_l] = np.repeat(-2*xi,N)[ind_l]
    except:
        pass
    return grad/N

# hessian of huber loss with respect to y_pred
def hess_huber_obj(y_true, y_pred):
    xi = 1.35
    y_true, y_pred = np.array(y_true).flatten(), np.array(y_pred).flatten()
    N = len(y_true)
    resid = y_true - y_pred
    ind_m = np.where(np.abs(resid)<=xi)
    ind_u = np.where(resid>xi)
    ind_l = np.where(resid< -xi)
    hess = np.zeros(N)
    try:
        hess[ind_m] = np.repeat(2,N)[ind_m]
    except:
        pass
    return hess/N

# huber loss for lgbm
def huber_obj(y_true, y_pred):
    grad = grad_huber_obj(y_true, y_pred)
    hess = hess_huber_obj(y_true, y_pred)
    return grad, hess

### 1.2.1 Random Forest

In [47]:
params = {
    'n_estimators': [300],
    'max_depth': [3, 6],
    'max_features': [30, 50, 100],
    'random_state': [12308]
}
RF = val_fun(RandomForestRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'max_depth': 3, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 2.45356%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 50, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 2.14061%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 100, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.77046%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 3.29298%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 50, 'n_estimators': 300, 'random_st

In [48]:
%%time

evaluate(y_trn, RF.predict(X_trn), insample=True) 
evaluate(y_tst, RF.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 31.20%
The in-sample MSE is 0.021
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 1.42%
The out-of-sample MSE is 0.040
CPU times: total: 1.08 s
Wall time: 1.08 s


### 1.2.2 Gradient Boosting (XGBRegression)

In [49]:
params = {
    'objective':[None, huber_obj],
    'max_depth':[1,2],
    'n_estimators':[500, 800, 1000],
    'random_state':[12308],
    'learning_rate':[.01,.1]
}
LGBM = val_fun(XGBRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 1.11101%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': <function huber_obj at 0x0000025EFD008430>, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: -0.19727%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 1.75432%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': <function huber_obj at 0x0000025EFD008430>, 'random_state': 12308} finished.
with out-of-sample R squared on validatio

In [50]:
%%time
evaluate(y_trn, LGBM.predict(X_trn), insample=True) 
evaluate(y_tst, LGBM.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 50.66%
The in-sample MSE is 0.015
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 18.57%
The out-of-sample MSE is 0.032
CPU times: total: 1.89 s
Wall time: 265 ms


In [51]:
output.iloc[0,6] = r2_score(y_tst, RF.predict(X_tst))
output.iloc[0,7] = r2_score(y_tst, LGBM.predict(X_tst))

### Diebold-Mariano tests

In [52]:
moriano.iloc[0,5] = dm_test(y_tst.values, OLS_H.predict(X_tst), RF.predict(X_tst), one_sided = False)[0]
moriano.iloc[0,6] = dm_test(y_tst.values, OLS_H.predict(X_tst), LGBM.predict(X_tst), one_sided = False)[0]

moriano.iloc[1,5] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), RF.predict(X_tst), one_sided = False)[0]
moriano.iloc[1,6] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), LGBM.predict(X_tst), one_sided = False)[0]

moriano.iloc[2,5] = dm_test(y_tst.values, PLS.predict(X_tst), RF.predict(X_tst), one_sided = False)[0]
moriano.iloc[2,6] = dm_test(y_tst.values, PLS.predict(X_tst), LGBM.predict(X_tst), one_sided = False)[0]

moriano.iloc[3,5] = dm_test(y_tst.values, PCR.predict(X_tst), RF.predict(X_tst), one_sided = False)[0]
moriano.iloc[3,6] = dm_test(y_tst.values, PCR.predict(X_tst), LGBM.predict(X_tst), one_sided = False)[0]

moriano.iloc[4,5] = dm_test(y_tst.values, EN_my_mse_rdm.predict(X_tst), RF.predict(X_tst), one_sided = False)[0]
moriano.iloc[4,6] = dm_test(y_tst.values, EN_my_mse_rdm.predict(X_tst), LGBM.predict(X_tst), one_sided = False)[0]

moriano.iloc[5,5] = dm_test(y_tst.values, GLM.predict(X_tst), RF.predict(X_tst), one_sided = False)[0]
moriano.iloc[5,6] = dm_test(y_tst.values, GLM.predict(X_tst), LGBM.predict(X_tst), one_sided = False)[0]

In [53]:
moriano

,OLS-3 + H,PLS,PCR,Enet + H,GLM + H,RF,GBRT + H,NN1,NN2,NN3,NN4,NN5
OLS + H,44.667607,13.617104,50.423308,47.538610,47.408230,36.402181,21.641169,0.0,0.0,0.0,0.0,0.0
OLS-3 + H,0.000000,-37.000430,-6.857542,21.822947,24.353913,11.264323,13.505666,0.0,0.0,0.0,0.0,0.0
PLS,0.000000,0.000000,42.498729,41.597149,41.618567,33.649792,20.598327,0.0,0.0,0.0,0.0,0.0
PCR,0.000000,0.000000,0.000000,20.385630,21.651692,12.197810,13.569594,0.0,0.0,0.0,0.0,0.0
Enet + H,0.000000,0.000000,0.000000,0.000000,17.784107,4.617354,12.171884,0.0,0.0,0.0,0.0,0.0
GLM + H,0.000000,0.000000,0.000000,0.000000,0.000000,3.408721,12.032375,0.0,0.0,0.0,0.0,0.0
RF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
GBRT + H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
NN1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
NN2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


### Coefficient Significance

In [54]:
significo['RF'] = RF.feature_importances_
significo['GBRT + H'] = LGBM.feature_importances_

In [55]:
significo

,PLS,PCR,Enet + H,GLM + H,RF,GBRT + H,NN1,NN2,NN3,NN4,NN5
pchgm_pchsale,-0.002017,-0.058591,0.0,[0.0],0.000892,0.000000,0.0,0.0,0.0,0.0,0.0
sic_80,0.000681,-0.000182,0.0,[0.0],0.000063,0.000000,0.0,0.0,0.0,0.0,0.0
sic_55,0.000914,-0.010684,0.0,[0.0],0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
currat,0.000368,0.036253,0.0,[0.0],0.003436,0.011125,0.0,0.0,0.0,0.0,0.0
sgr,-0.000459,0.052028,0.0,[-0.0],0.003557,0.001681,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
sic_81,0.000663,-0.002312,0.0,[-0.0],0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
ill,0.004975,0.014122,0.0,[0.0],0.007285,0.008632,0.0,0.0,0.0,0.0,0.0
sic_49,0.000419,-0.026146,0.0,[0.0],0.000023,0.000000,0.0,0.0,0.0,0.0,0.0
age,-0.001054,-0.116730,0.0,[0.0],0.000656,0.000000,0.0,0.0,0.0,0.0,0.0


In [97]:
significo['PLS'].sort_values(ascending = False)

baspread      0.009017
sic_73        0.007239
ill           0.004975
maxret        0.004049
sic_38        0.003435
                ...   
chpmia       -0.005962
std_turn     -0.006613
std_dolvol   -0.007444
mom1m        -0.012592
indmom       -0.018105
Name: PLS, Length: 169, dtype: float64

## 1.3 Neural Networks

In [56]:
# customized metrics
# out-of-sample r squared for keras
def R_oos_tf(y_true, y_pred):
    resid = tf.square(y_true-y_pred)
    denom = tf.square(y_true)
    return 1 - tf.divide(tf.reduce_mean(resid),tf.reduce_mean(denom))

# data standardization
# please standardize the data if BatchNormalization is not used
def standardize(X_trn, X_vld, X_tst):
    mu_trn = np.mean(np.array(X_trn),axis=0).reshape((1,X_trn.shape[1]))
    sigma_trn = np.std(np.array(X_trn),axis=0).reshape((1,X_trn.shape[1]))

    X_trn_std = (np.array(X_trn)-mu_trn)/sigma_trn
    X_vld_std = (np.array(X_vld)-mu_trn)/sigma_trn
    X_tst_std = (np.array(X_tst)-mu_trn)/sigma_trn
    return X_trn_std, X_vld_std, X_tst_std

# NN class
class NN:
    
    def __init__(
        self, n_layers=1, loss='mse', l1=1e-5, l2=0, learning_rate=.01, BatchNormalization=True, patience=5,
        epochs=100, batch_size=3000, verbose=1, random_state=12308, monitor='val_R_oos_tf', base_neurons=5
    ):
        self.n_layers = n_layers
        self.l1 = l1
        self.l2 = l2
        self.learning_rate = learning_rate
        self.BatchNormalization = BatchNormalization
        self.patience = patience
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.random_state = random_state
        self.monitor = monitor
        self.base_neurons = base_neurons

    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
    
    def fit(self, X_trn, y_trn, X_vld, y_vld):
        # fix random seed for reproducibility
        random.seed(self.random_state)
        np.random.seed(self.random_state)
        tf.random.set_seed(self.random_state)
        
        # model construction
        mod = Sequential()
        mod.add(Input(shape=(X_trn.shape[1],)))
        
        for i in np.arange(self.n_layers,0,-1):
            if self.n_layers>self.base_neurons:
                if self.n_layers == i:
                    mod.add(Dense(2**i, activation='relu'))
                else:
                    mod.add(Dense(2**i, activation='relu', kernel_regularizer=L1L2(self.l1,self.l2)))
            else:
                if self.n_layers == i:
                    mod.add(Dense(2**(self.base_neurons-(self.n_layers-i)), activation='relu'))
                else:
                    mod.add(Dense(2**(self.base_neurons-(self.n_layers-i)), 
                                  activation='relu', kernel_regularizer=L1L2(self.l1,self.l2)))
            if self.BatchNormalization:
                mod.add(BatchNormalization())
        
        mod.add(Dense(1, kernel_regularizer=L1L2(self.l1,self.l2)))
        
        # early stopping
        earlystop = tf.keras.callbacks.EarlyStopping(monitor=self.monitor, patience=self.patience, mode = 'max')

        # Adam solver
        opt = Adam(learning_rate=self.learning_rate)
        
        # compile the model
        mod.compile(loss=self.loss,
                    optimizer=opt,
                    metrics=[R_oos_tf])

        # fit the model
        mod.fit(X_trn, np.array(y_trn).reshape((len(y_trn),1)), epochs=self.epochs, batch_size=self.batch_size, 
                callbacks=[earlystop], verbose=self.verbose, 
                validation_data=(X_vld,np.array(y_vld).reshape((len(y_vld),1))))
        
        self.model = mod
        return self
    
    def predict(self, X):
        return self.model.predict(X, verbose=self.verbose)

In [57]:
%%time
# NN1-Regression-[32(relu)-1(linear)]

params = {
    'n_layers': [1],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf'],
}
NN1 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -119.88697%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 0.26530%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 2.427

In [58]:
%%time
# NN2-Regression-[32(relu)-16(relu)-1(linear)]
params = {
    'n_layers': [2],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN2 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 2, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -184.38131%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 2, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 0.00880%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 2, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -73.6

In [59]:
%%time
# NN3-Regression-[32(relu)-16(relu)-8(relu)-1(linear)]
params = {
    'n_layers': [3],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN3 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 3, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -9.27175%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 3, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 2.80203%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 3, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 0.41440

In [60]:
%%time
# NN4-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-1(linear)]
params = {
    'n_layers': [4],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN4 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 4, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -12.60764%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 4, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -8.48823%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 4, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -0.25

In [61]:
%%time
# NN5-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-2(relu)-1(linear)]
params = {
    'n_layers': [5],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN5 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 5, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 0.08453%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 5, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -0.48976%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1366, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 5, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -0.0258

In [62]:
output['NN1'] = np.array([0,0,0])
output['NN2'] = np.array([0,0,0])
output['NN3'] = np.array([0,0,0])
output['NN4'] = np.array([0,0,0])
output['NN5'] = np.array([0,0,0])
output.iloc[0,8] = r2_score(y_tst, NN1.predict(X_tst))
output.iloc[0,9] = r2_score(y_tst, NN2.predict(X_tst))
output.iloc[0,10] = r2_score(y_tst, NN3.predict(X_tst))
output.iloc[0,11] = r2_score(y_tst, NN4.predict(X_tst))
output.iloc[0,12] = r2_score(y_tst, NN5.predict(X_tst))

### Diebold-Mariano tests

In [63]:
moriano.iloc[0,7] = dm_test(y_tst.values, OLS_H.predict(X_tst), NN1.predict(X_tst), one_sided = False)[0]
moriano.iloc[0,8] = dm_test(y_tst.values, OLS_H.predict(X_tst), NN2.predict(X_tst), one_sided = False)[0]
moriano.iloc[0,9] = dm_test(y_tst.values, OLS_H.predict(X_tst), NN3.predict(X_tst), one_sided = False)[0]
moriano.iloc[0,10] = dm_test(y_tst.values, OLS_H.predict(X_tst), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[0,11] = dm_test(y_tst.values, OLS_H.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]

moriano.iloc[1,7] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), NN1.predict(X_tst), one_sided = False)[0]
moriano.iloc[1,8] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), NN2.predict(X_tst), one_sided = False)[0]
moriano.iloc[1,9] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), NN3.predict(X_tst), one_sided = False)[0]
moriano.iloc[1,10] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[1,11] = dm_test(y_tst.values, OLS_3.predict(X_tst[features_3]), NN5.predict(X_tst), one_sided = False)[0]

moriano.iloc[2,7] = dm_test(y_tst.values, PLS.predict(X_tst), NN1.predict(X_tst), one_sided = False)[0]
moriano.iloc[2,8] = dm_test(y_tst.values, PLS.predict(X_tst), NN2.predict(X_tst), one_sided = False)[0]
moriano.iloc[2,9] = dm_test(y_tst.values, PLS.predict(X_tst), NN3.predict(X_tst), one_sided = False)[0]
moriano.iloc[2,10] = dm_test(y_tst.values, PLS.predict(X_tst), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[2,11] = dm_test(y_tst.values, PLS.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]

moriano.iloc[3,7] = dm_test(y_tst.values, PCR.predict(X_tst), NN1.predict(X_tst), one_sided = False)[0]
moriano.iloc[3,8] = dm_test(y_tst.values, PCR.predict(X_tst), NN2.predict(X_tst), one_sided = False)[0]
moriano.iloc[3,9] = dm_test(y_tst.values, PCR.predict(X_tst), NN3.predict(X_tst), one_sided = False)[0]
moriano.iloc[3,10] = dm_test(y_tst.values, PCR.predict(X_tst), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[3,11] = dm_test(y_tst.values, PCR.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]

moriano.iloc[4,7] = dm_test(y_tst.values, EN_my_mse_rdm.predict(X_tst), NN1.predict(X_tst), one_sided = False)[0]
moriano.iloc[4,8] = dm_test(y_tst.values, EN_my_mse_rdm.predict(X_tst), NN2.predict(X_tst), one_sided = False)[0]
moriano.iloc[4,9] = dm_test(y_tst.values, EN_my_mse_rdm.predict(X_tst), NN3.predict(X_tst), one_sided = False)[0]
moriano.iloc[4,10] = dm_test(y_tst.values, EN_my_mse_rdm.predict(X_tst), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[4,11] = dm_test(y_tst.values, EN_my_mse_rdm.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]

moriano.iloc[5,7] = dm_test(y_tst.values, GLM.predict(X_tst), NN1.predict(X_tst), one_sided = False)[0]
moriano.iloc[5,8] = dm_test(y_tst.values, GLM.predict(X_tst), NN2.predict(X_tst), one_sided = False)[0]
moriano.iloc[5,9] = dm_test(y_tst.values, GLM.predict(X_tst), NN3.predict(X_tst), one_sided = False)[0]
moriano.iloc[5,10] = dm_test(y_tst.values, GLM.predict(X_tst), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[5,11] = dm_test(y_tst.values, GLM.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]

moriano.iloc[6,7] = dm_test(y_tst.values, RF.predict(X_tst), NN1.predict(X_tst), one_sided = False)[0]
moriano.iloc[6,8] = dm_test(y_tst.values, RF.predict(X_tst), NN2.predict(X_tst), one_sided = False)[0]
moriano.iloc[6,9] = dm_test(y_tst.values, RF.predict(X_tst), NN3.predict(X_tst), one_sided = False)[0]
moriano.iloc[6,10] = dm_test(y_tst.values, RF.predict(X_tst), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[6,11] = dm_test(y_tst.values, RF.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]

moriano.iloc[7,7] = dm_test(y_tst.values, LGBM.predict(X_tst), NN1.predict(X_tst), one_sided = False)[0]
moriano.iloc[7,8] = dm_test(y_tst.values, LGBM.predict(X_tst), NN2.predict(X_tst), one_sided = False)[0]
moriano.iloc[7,9] = dm_test(y_tst.values, LGBM.predict(X_tst), NN3.predict(X_tst), one_sided = False)[0]
moriano.iloc[7,10] = dm_test(y_tst.values, LGBM.predict(X_tst), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[7,11] = dm_test(y_tst.values, LGBM.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]


moriano.iloc[8,8] = dm_test(y_tst.values, NN1.predict(X_tst), NN2.predict(X_tst), one_sided = False)[0]
moriano.iloc[8,9] = dm_test(y_tst.values, NN1.predict(X_tst), NN3.predict(X_tst), one_sided = False)[0]
moriano.iloc[8,10] = dm_test(y_tst.values, NN1.predict(X_tst), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[8,11] = dm_test(y_tst.values, NN1.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]

moriano.iloc[9,9] = dm_test(y_tst.values, NN2.predict(X_tst), NN3.predict(X_tst), one_sided = False)[0]
moriano.iloc[9,10] = dm_test(y_tst.values, NN2.predict(X_tst), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[9,11] = dm_test(y_tst.values, NN2.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]

moriano.iloc[10,10] = dm_test(y_tst.values, NN3.predict(X_tst), NN4.predict(X_tst), one_sided = False)[0]
moriano.iloc[10,11] = dm_test(y_tst.values, NN3.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]

moriano.iloc[11,11] = dm_test(y_tst.values, NN4.predict(X_tst), NN5.predict(X_tst), one_sided = False)[0]

In [64]:
moriano

,OLS-3 + H,PLS,PCR,Enet + H,GLM + H,RF,GBRT + H,NN1,NN2,NN3,NN4,NN5
OLS + H,44.667607,13.617104,50.423308,47.538610,47.408230,36.402181,21.641169,-11.697275,-49.387508,-0.363544,46.780575,49.806442
OLS-3 + H,0.000000,-37.000430,-6.857542,21.822947,24.353913,11.264323,13.505666,-56.849159,-80.963097,-31.158699,2.877297,-15.552996
PLS,0.000000,0.000000,42.498729,41.597149,41.618567,33.649792,20.598327,-28.597974,-45.142408,-9.796310,39.354866,38.316115
PCR,0.000000,0.000000,0.000000,20.385630,21.651692,12.197810,13.569594,-66.263751,-86.115869,-34.302898,7.311327,-10.938444
Enet + H,0.000000,0.000000,0.000000,0.000000,17.784107,4.617354,12.171884,-60.745200,-85.063556,-34.177638,-15.006216,-33.213926
GLM + H,0.000000,0.000000,0.000000,0.000000,0.000000,3.408721,12.032375,-60.085903,-84.415741,-34.179653,-17.615084,-33.122356
RF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-44.734188,-65.644951,-29.271791,-10.572270,-16.893509
GBRT + H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-23.371767,-30.534307,-20.389906,-13.148064,-14.411870
NN1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-47.033726,4.256386,59.759953,68.758141
NN2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.243170,85.851082,97.941765


## 1.3 Top

In [65]:
b1_top = b1_d.sort_values('mvel1',ascending=False).groupby('date').head(1000)

In [66]:
index_co = b1_top.index
features = list(set(b1_top.columns).difference({'permno','DATE','RET'}))
X = MinMaxScaler((-1,1)).fit_transform(b1_top[features])
X = pd.DataFrame(X, columns=features, index=index_co)
y = b1_top['RET']

stdt_vld = starting + (ending - starting)/3
stdt_tst = starting + 2*(ending - starting)/3

X_trn = X[X.index.get_level_values(level = "date") <= stdt_vld]
y_trn = y[y.index.get_level_values(level = "date") <= stdt_vld]

X_vld = X[(X.index.get_level_values(level = "date") >= stdt_vld) & (X.index.get_level_values(level = "date") <= stdt_tst)]
y_vld = y[(y.index.get_level_values(level = "date") >= stdt_vld) & (y.index.get_level_values(level = "date") <= stdt_tst)]

X_tst = X[X.index.get_level_values(level = "date") >= stdt_tst]
y_tst = y[y.index.get_level_values(level = "date") >= stdt_tst]

In [67]:
epsilon = np.max(((y_trn-OLS.predict(X_trn)).quantile(.999),1))
OLS_H = HuberRegressor(epsilon=epsilon).fit(X_trn,y_trn)

params = {'n_components': [1, 5, 10, 50]}
PLS = val_fun(PLSRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

params = {'n_PCs':[1,3,5,7,10,50],'loss':['mse','huber']}
PCR = val_fun(PCRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,sleep=3)

EN_my_mse_rdm = ENet(lmd=.01,rho=.5,loss='huber').fit(X_trn,y_trn)

params = {
    'knots':[3],
    'lmd':[1e-4,1e-1],#list(np.linspace(1e-4,1e-1,10)),
    'l1_reg':[1e-4,0]
}
GLM = val_fun(GLMRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']
OLS_3 = LinearRegression().fit(X_trn[features_3],y_trn)

Model with params: {'n_components': 1} finished.
with out-of-sample R squared on validation set: 6.91720%
************************************************************
Model with params: {'n_components': 5} finished.
with out-of-sample R squared on validation set: 4.52028%
************************************************************
Model with params: {'n_components': 10} finished.
with out-of-sample R squared on validation set: 2.86558%
************************************************************
Model with params: {'n_components': 50} finished.
with out-of-sample R squared on validation set: 3.51104%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'n_components': 1}
with R2oos 6.92% on validation set.
############################################################
Model with params: {'loss': 'mse', 'n_PCs': 1} finished.
with out-of-sample R squared on validation set

In [68]:
output.iloc[1,0] =r2_score(y_tst, OLS_H.predict(X_tst))
output.iloc[1,2] = r2_score(y_tst, PCR.predict(X_tst))
output.iloc[1,3] = r2_score(y_tst, PLS.predict(X_tst))
output.iloc[1,4] = r2_score(y_tst, EN_my_mse_rdm.predict(X_tst))
output.iloc[1,5] = r2_score(y_tst, GLM.predict(X_tst))

output.iloc[1,1] = r2_score(y_tst, OLS_3.predict(X_tst[features_3]))

In [69]:
params = {
    'n_estimators': [300],
    'max_depth': [3, 6],
    'max_features': [30, 50, 100],
    'random_state': [12308]
}
RF = val_fun(RandomForestRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'max_depth': 3, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.73180%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 50, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: -0.70376%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 100, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: -5.41997%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.69172%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 50, 'n_estimators': 300, 'random_

In [70]:
params = {
    'objective':[None, huber_obj],
    'max_depth':[1,2],
    'n_estimators':[500, 800, 1000],
    'random_state':[12308],
    'learning_rate':[.01,.1]
}
LGBM = val_fun(XGBRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: -4.72057%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': <function huber_obj at 0x0000025EFD008430>, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: -2.72356%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: -3.75069%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': <function huber_obj at 0x0000025EFD008430>, 'random_state': 12308} finished.
with out-of-sample R squared on validat

In [71]:
output.iloc[1,6] = r2_score(y_tst, RF.predict(X_tst))
output.iloc[1,7] = r2_score(y_tst, LGBM.predict(X_tst))

In [72]:
%%time
# NN1-Regression-[32(relu)-1(linear)]

params = {
    'n_layers': [1],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf'],
}
NN1 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN2-Regression-[32(relu)-16(relu)-1(linear)]
params = {
    'n_layers': [2],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN2 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN3-Regression-[32(relu)-16(relu)-8(relu)-1(linear)]
params = {
    'n_layers': [3],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN3 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN4-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-1(linear)]
params = {
    'n_layers': [4],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN4 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN5-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-2(relu)-1(linear)]
params = {
    'n_layers': [5],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN5 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 240, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -294.29593%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 240, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -30.53179%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 240, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 1.8796

UsageError: Line magic function `%%time` not found.


In [73]:
output.iloc[1,8] = r2_score(y_tst, NN1.predict(X_tst))
output.iloc[1,9] = r2_score(y_tst, NN2.predict(X_tst))
output.iloc[1,10] = r2_score(y_tst, NN3.predict(X_tst))
output.iloc[1,11] = r2_score(y_tst, NN4.predict(X_tst))
output.iloc[1,12] = r2_score(y_tst, NN5.predict(X_tst))

## 1.5 Bottom

In [74]:
b1_bot = b1_d.sort_values('mvel1',ascending=False).groupby('date').tail(1000)

In [75]:
index_co = b1_bot.index
features = list(set(b1_bot.columns).difference({'permno','DATE','RET'}))
X = MinMaxScaler((-1,1)).fit_transform(b1_bot[features])
X = pd.DataFrame(X, columns=features, index=index_co)
y = b1_bot['RET']

stdt_vld = starting + (ending - starting)/3
stdt_tst = starting + 2*(ending - starting)/3

X_trn = X[X.index.get_level_values(level = "date") <= stdt_vld]
y_trn = y[y.index.get_level_values(level = "date") <= stdt_vld]

X_vld = X[(X.index.get_level_values(level = "date") >= stdt_vld) & (X.index.get_level_values(level = "date") <= stdt_tst)]
y_vld = y[(y.index.get_level_values(level = "date") >= stdt_vld) & (y.index.get_level_values(level = "date") <= stdt_tst)]

X_tst = X[X.index.get_level_values(level = "date") >= stdt_tst]
y_tst = y[y.index.get_level_values(level = "date") >= stdt_tst]

In [76]:
epsilon = np.max(((y_trn-OLS.predict(X_trn)).quantile(.999),1))
OLS_H = HuberRegressor(epsilon=epsilon).fit(X_trn,y_trn)

params = {'n_components': [1, 5, 10, 50]}
PLS = val_fun(PLSRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

params = {'n_PCs':[1,3,5,7,10,50],'loss':['mse','huber']}
PCR = val_fun(PCRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,sleep=3)

EN_my_mse_rdm = ENet(lmd=.01,rho=.5,loss='huber').fit(X_trn,y_trn)

params = {
    'knots':[3],
    'lmd':[1e-4,1e-1],#list(np.linspace(1e-4,1e-1,10)),
    'l1_reg':[1e-4,0]
}
GLM = val_fun(GLMRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']
OLS_3 = LinearRegression().fit(X_trn[features_3],y_trn)

Model with params: {'n_components': 1} finished.
with out-of-sample R squared on validation set: 1.18813%
************************************************************
Model with params: {'n_components': 5} finished.
with out-of-sample R squared on validation set: 3.50014%
************************************************************
Model with params: {'n_components': 10} finished.
with out-of-sample R squared on validation set: 3.54722%
************************************************************
Model with params: {'n_components': 50} finished.
with out-of-sample R squared on validation set: 3.53315%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'n_components': 10}
with R2oos 3.55% on validation set.
############################################################
Model with params: {'loss': 'mse', 'n_PCs': 1} finished.
with out-of-sample R squared on validation se

In [77]:
output.iloc[2,0] =r2_score(y_tst, OLS_H.predict(X_tst))
output.iloc[2,2] = r2_score(y_tst, PCR.predict(X_tst))
output.iloc[2,3] = r2_score(y_tst, PLS.predict(X_tst))
output.iloc[2,4] = r2_score(y_tst, EN_my_mse_rdm.predict(X_tst))
output.iloc[2,5] = r2_score(y_tst, GLM.predict(X_tst))

output.iloc[2,1] = r2_score(y_tst, OLS_3.predict(X_tst[features_3]))

In [78]:
params = {
    'n_estimators': [300],
    'max_depth': [3, 6],
    'max_features': [30, 50, 100],
    'random_state': [12308]
}
RF = val_fun(RandomForestRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'max_depth': 3, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 8.14864%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 50, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 9.92383%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 100, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 13.10519%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 13.02828%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 50, 'n_estimators': 300, 'random_

In [79]:
params = {
    'objective':[None, huber_obj],
    'max_depth':[1,2],
    'n_estimators':[500, 800, 1000],
    'random_state':[12308],
    'learning_rate':[.01,.1]
}
LGBM = val_fun(XGBRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 7.01012%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'objective': <function huber_obj at 0x0000025EFD008430>, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.20518%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 8.14844%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'objective': <function huber_obj at 0x0000025EFD008430>, 'random_state': 12308} finished.
with out-of-sample R squared on validation

In [80]:
output.iloc[2,6] = r2_score(y_tst, RF.predict(X_tst))
output.iloc[2,7] = r2_score(y_tst, LGBM.predict(X_tst))

In [81]:
%%time
# NN1-Regression-[32(relu)-1(linear)]

params = {
    'n_layers': [1],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf'],
}
NN1 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN2-Regression-[32(relu)-16(relu)-1(linear)]
params = {
    'n_layers': [2],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN2 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN3-Regression-[32(relu)-16(relu)-8(relu)-1(linear)]
params = {
    'n_layers': [3],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN3 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN4-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-1(linear)]
params = {
    'n_layers': [4],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN4 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

%%time
# NN5-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-2(relu)-1(linear)]
params = {
    'n_layers': [5],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN5 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 240, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 2.65815%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 240, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 3.99095%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 240, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 2.56912%
**

UsageError: Line magic function `%%time` not found.


In [82]:
output.iloc[2,8] = r2_score(y_tst, NN1.predict(X_tst))
output.iloc[2,9] = r2_score(y_tst, NN2.predict(X_tst))
output.iloc[2,10] = r2_score(y_tst, NN3.predict(X_tst))
output.iloc[2,11] = r2_score(y_tst, NN4.predict(X_tst))
output.iloc[2,12] = r2_score(y_tst, NN5.predict(X_tst))

In [83]:
output.to_excel("reality_here.xlsx")

In [84]:
output

,OLS + H,OLS-3 + H,PCR,PLS,Enet + H,GLM + H,RF,GBRT + H,NN1,NN2,NN3,NN4,NN5
All,-0.258922,-0.042222,-0.051462,-0.223856,-0.013536,-0.009523,0.000526,0.201412,-0.278784,-0.420429,-0.260544,-0.036583,-0.068679
Top 1000,-0.518628,-0.118402,-0.099896,-0.083866,-0.034174,-0.028305,0.002884,0.136232,-1.449462,-1.488644,-2.641036,-0.080898,-0.245133
Bottom 1000,-0.215506,-0.034741,-0.046066,-0.145577,-0.005900,-0.005751,0.122885,0.184621,-0.190637,-0.226046,-0.150685,-0.016387,-0.037959


In [85]:
significo.to_excel("model_significance.xlsx")

In [86]:
significo

,PLS,PCR,Enet + H,GLM + H,RF,GBRT + H,NN1,NN2,NN3,NN4,NN5
pchgm_pchsale,-0.002017,-0.058591,0.0,[0.0],0.000892,0.000000,0.0,0.0,0.0,0.0,0.0
sic_80,0.000681,-0.000182,0.0,[0.0],0.000063,0.000000,0.0,0.0,0.0,0.0,0.0
sic_55,0.000914,-0.010684,0.0,[0.0],0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
currat,0.000368,0.036253,0.0,[0.0],0.003436,0.011125,0.0,0.0,0.0,0.0,0.0
sgr,-0.000459,0.052028,0.0,[-0.0],0.003557,0.001681,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
sic_81,0.000663,-0.002312,0.0,[-0.0],0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
ill,0.004975,0.014122,0.0,[0.0],0.007285,0.008632,0.0,0.0,0.0,0.0,0.0
sic_49,0.000419,-0.026146,0.0,[0.0],0.000023,0.000000,0.0,0.0,0.0,0.0,0.0
age,-0.001054,-0.116730,0.0,[0.0],0.000656,0.000000,0.0,0.0,0.0,0.0,0.0


In [88]:
moriano.to_excel("dieboldmariano_test.xlsx")

In [89]:
moriano

,OLS-3 + H,PLS,PCR,Enet + H,GLM + H,RF,GBRT + H,NN1,NN2,NN3,NN4,NN5
OLS + H,44.667607,13.617104,50.423308,47.538610,47.408230,36.402181,21.641169,-11.697275,-49.387508,-0.363544,46.780575,49.806442
OLS-3 + H,0.000000,-37.000430,-6.857542,21.822947,24.353913,11.264323,13.505666,-56.849159,-80.963097,-31.158699,2.877297,-15.552996
PLS,0.000000,0.000000,42.498729,41.597149,41.618567,33.649792,20.598327,-28.597974,-45.142408,-9.796310,39.354866,38.316115
PCR,0.000000,0.000000,0.000000,20.385630,21.651692,12.197810,13.569594,-66.263751,-86.115869,-34.302898,7.311327,-10.938444
Enet + H,0.000000,0.000000,0.000000,0.000000,17.784107,4.617354,12.171884,-60.745200,-85.063556,-34.177638,-15.006216,-33.213926
GLM + H,0.000000,0.000000,0.000000,0.000000,0.000000,3.408721,12.032375,-60.085903,-84.415741,-34.179653,-17.615084,-33.122356
RF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-44.734188,-65.644951,-29.271791,-10.572270,-16.893509
GBRT + H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-23.371767,-30.534307,-20.389906,-13.148064,-14.411870
NN1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-47.033726,4.256386,59.759953,68.758141
NN2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.243170,85.851082,97.941765


In [ ]:
### Tursday Morning 2nd of May 8:00 a.m. => 10 p.m Wednesday my time